In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
df = pd.read_csv('/Users/hrishityelchuri/Desktop/github/grabhack/driver-credit-scoring-engine/data/processed/df_clean.csv')

In [55]:
df.sample(10)

,partner_id,role,gender,age_group,city_tier,tenure_months,days_active,trips_per_week,earnings_avg,earnings_var,...,cancel_rate,customer_rating,complaints,accidents,night_shift_pct,cashless_ratio,wallet_txn_volume,vehicle_age,past_due_history,good_repayment
5529,5530,driver,M,36-50,3,12,25,80,374.461889,3.725856,...,0.085374,4.736284,0,0,0.120449,0.580166,256.033941,3.229792,0,1
628,629,driver,M,26-35,3,33,23,58,936.899255,4.976209,...,0.161055,4.620026,0,0,0.165559,0.871366,285.471094,3.053724,0,1
7245,7246,driver,M,50+,2,14,24,74,670.645794,5.883610,...,0.034735,4.996667,1,0,0.100017,0.929838,289.227191,6.432317,0,1
2621,2622,merchant,M,36-50,1,26,7,79,359.135242,5.931666,...,0.111465,4.612387,1,0,0.422260,0.721547,381.983678,1.671067,0,1
5541,5542,driver,F,26-35,1,16,30,23,948.392638,3.952082,...,0.123985,4.928072,1,0,0.564692,0.526345,166.493661,5.763483,0,1
7446,7447,driver,M,26-35,2,40,19,79,728.403055,4.271601,...,0.174838,4.707721,1,0,0.240256,0.492761,257.821475,5.886978,0,0
6496,6497,driver,F,36-50,2,36,21,58,897.020366,4.686588,...,0.177989,4.859725,0,0,0.267576,0.545810,221.432219,3.277599,0,1
7402,7403,driver,F,26-35,3,9,14,32,499.505954,4.254595,...,0.078297,4.933088,1,0,0.417875,0.736202,184.706598,5.048294,0,1
2579,2580,driver,F,26-35,2,25,26,41,978.239328,4.144770,...,0.114425,4.718521,1,0,0.365272,0.712034,197.639583,6.074646,0,1
6011,6012,merchant,F,36-50,3,22,13,53,688.998552,4.608149,...,0.041963,5.000000,0,0,0.203601,0.824923,214.710772,4.474459,0,1


In [56]:
df['vehicle_age']

0       4.336435
1       5.482631
2       6.072208
3       3.909398
4       3.512688
          ...   
7995    4.551144
7996    0.000000
7997    1.850392
7998    4.547698
7999    5.206713
Name: vehicle_age, Length: 8000, dtype: float64

In [57]:
df["vehicle_age_bin"] = pd.cut(df["vehicle_age"], 
                               bins=[0, 2, 5, 10, 20], 
                               labels=["new(0-2)", "mid(3-5)", "old(6-10)", "very_old(10+)"])

In [58]:
df["earnings_cv"] = df["earnings_var"] / (df["earnings_avg"] + 1e-6)

In [59]:
df["productivity_stability"] = df["trips_per_week"] / df["days_active"]

In [60]:
# Feature engineering
df["earnings_stability"] = df["earnings_avg"] / (1 + df["earnings_var"])
df["wallet_txn_bin"] = pd.cut(df["wallet_txn_volume"], bins=[-1, 0, 2, 100], labels=["0", "1-2", "3+"])

In [61]:
df["tenure_x_earnings"] = df["tenure_months"] * df["earnings_avg"]
df["active_x_reliability"] = df["days_active"] * (1 - df["cancel_rate"])

In [62]:
df.columns

Index(['partner_id', 'role', 'gender', 'age_group', 'city_tier',
       'tenure_months', 'days_active', 'trips_per_week', 'earnings_avg',
       'earnings_var', 'on_time_rate', 'cancel_rate', 'customer_rating',
       'complaints', 'accidents', 'night_shift_pct', 'cashless_ratio',
       'wallet_txn_volume', 'vehicle_age', 'past_due_history',
       'good_repayment', 'vehicle_age_bin', 'earnings_cv',
       'productivity_stability', 'earnings_stability', 'wallet_txn_bin',
       'tenure_x_earnings', 'active_x_reliability'],
      dtype='object')

In [63]:
df.head(5)

,partner_id,role,gender,age_group,city_tier,tenure_months,days_active,trips_per_week,earnings_avg,earnings_var,...,vehicle_age,past_due_history,good_repayment,vehicle_age_bin,earnings_cv,productivity_stability,earnings_stability,wallet_txn_bin,tenure_x_earnings,active_x_reliability
0,1,driver,M,<=25,2,36,19,52,638.443420,4.633046,...,4.336435,0,1,mid(3-5),0.007257,2.736842,113.338925,NaN,22983.963112,16.419894
1,2,merchant,M,<=25,2,17,24,53,444.440684,4.250656,...,5.482631,0,1,old(6-10),0.009564,2.208333,84.644785,NaN,7555.491634,22.528292
2,3,merchant,M,26-35,1,61,28,35,659.666189,5.412836,...,6.072208,0,1,old(6-10),0.008205,1.250000,102.866529,NaN,40239.637528,26.722965
3,4,driver,F,<=25,3,33,18,58,503.720971,3.237181,...,3.909398,0,0,mid(3-5),0.006427,3.222222,118.881158,NaN,16622.792045,17.036618
4,5,driver,M,50+,3,27,19,57,637.742060,5.022423,...,3.512688,0,1,mid(3-5),0.007875,3.000000,105.894600,0,17219.035633,16.140018


In [64]:
df.drop(columns = ["vehicle_age", "trips_per_week", "earnings_var", "tenure_months", "days_active"], inplace = True)

In [65]:
df.to_csv("/Users/hrishityelchuri/Desktop/github/grabhack/driver-credit-scoring-engine/data/processed/df_feature_engineered.csv", index=False)